# Exercise 04 - Due Friday, October 21 at 12pm

*Objectives*: Gain experience loading a CSV dataset into a database model you define yourself and using SQL to explore its contents. Explore the data by writing and executing a number of SQL queries using common syntax and functions and describing your findings.

*Grading criteria*: The tasks should all be completed, and questions should all be answered with SQL queries in the space provided, unless a text answer is requested. Results should be correct, according to the embedded tests. The notebook itself should be completely reproducible; from start to finish, another person should be able to use the same code to obtain the same results as yours.  Note that you will receive no more than partial credit if you do not add text/markdown cells explaining your thinking at each major step in each problem.

*Deadline*: Friday, October 21, 12pm. Submit your notebook to Blackboard and push it to your GitHub repository.

## Setup - obtain data and create database

The US City Open Data Census has data on [restaurant inspections from all over the country](http://us-city.census.okfn.org/dataset/food-safety).  Let's take a look at data from Albuquerque.  *Note*: metadata and other details for this dataset are available at http://data.cabq.gov/business/foodinspections/.

Fyi, to save you a step, I've converted this file into a clean CSV file by using the `dos2unix` command to change its line endings and to switch it to use commas instead of tabs using `csvformat -t inputfile.csv > outputfile.csv`.

First we download the dataset:

In [1]:
!wget https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/exercises/abq-food-inspections-fy2010.csv

--2016-10-24 14:16:38--  https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/exercises/abq-food-inspections-fy2010.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.32.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.32.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5025226 (4.8M) [text/plain]
Saving to: ‘abq-food-inspections-fy2010.csv’

abq-food-inspection 100%[=====================>]   4.79M  --.-KB/s   in 0.1s   

2016-10-24 14:16:38 (45.9 MB/s) - ‘abq-food-inspections-fy2010.csv’ saved [5025226/5025226]



## Problem 1 - examine the data (35 points)

Use `csvstat` and other tools as you see fit and observe its contents.  Review the metadata definitions linked to above.  Describe what you see:  What do all the columns mean?  Are there null values?  Which columns interest you the most?  Which columns present some complications or questions you would like to ask?

In [2]:
%load_ext sql

/opt/conda/lib/python3.5/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/opt/conda/lib/python3.5/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [3]:
!echo 'redspot' | sudo -S service postgresql restart

[sudo] password for jovyan: Restarting PostgreSQL 9.5 database server: main.


In [4]:
!createdb -U dbuser cbunker

createdb: database creation failed: ERROR:  database "cbunker" already exists


In [5]:
%sql postgresql://dbuser@localhost:5432/cbunker

'Connected: dbuser@cbunker'

In [6]:
!mv abq-food-inspections-fy2010.csv inspection.csv

In [7]:
!csvcut -n inspection.csv

  1: FACILITY_NAME
  2: FACILITY_KEY
  3: SITE_ADDRESS
  4: CITY
  5: STATE
  6: ZIP
  7: OWNER_KEY
  8: OWNER_NAME
  9: NATURE_OF_BUSINESS
 10: STREET_NUMBER
 11: STREET_NAME
 12: STREET_TYPE
 13: POST_DIRECTIONAL
 14: PHONE
 15: PROGRAM_CATEGORY
 16: PROGRAM_CATEGORY_DESCRIPTION
 17: INSPECTION_DATE
 18: INSPECTION_TYPE
 19: INSPECTION_DESC
 20: SERIAL_NUM
 21: ACTION_CODE
 22: ACTION_DESC
 23: RESULT_CODE
 24: RESULT_DESC
 25: VIOLATION_CODE
 26: VIOLATION_DESC
 27: INSPECTION_MEMO


From above we can see the amount of fields and their names. 

In [8]:
!head inspection.csv | csvlook

|--------------------------+--------------+-------------------------------+-------------+-------+-------+-----------+-------------------------+--------------------+---------------+-------------+-------------+------------------+------------+------------------+------------------------------+---------------------+-----------------+-------------------------+------------+-------------+-------------+-------------+----------------+----------------+----------------------------------+------------------|
|  FACILITY_NAME           | FACILITY_KEY | SITE_ADDRESS                  | CITY        | STATE | ZIP   | OWNER_KEY | OWNER_NAME              | NATURE_OF_BUSINESS | STREET_NUMBER | STREET_NAME | STREET_TYPE | POST_DIRECTIONAL | PHONE      | PROGRAM_CATEGORY | PROGRAM_CATEGORY_DESCRIPTION | INSPECTION_DATE     | INSPECTION_TYPE | INSPECTION_DESC         | SERIAL_NUM | ACTION_CODE | ACTION_DESC | RESULT_CODE | RESULT_DESC    | VIOLATION_CODE | VIOLATION_DESC                   | INSPECTION_MEMO  |


In [9]:
!csvstat inspection.csv

  1. FACILITY_NAME
	<class 'str'>
	Nulls: False
	Unique values: 2615
	5 most frequent values:
		SMITHS FOOD AND DRUG CENTERS INC:	154
		ISOTOPES PARK - OVATIONS:	93
		PER EHSII CECELIA GARCIA OOB LA REYNA DEL SUR:	85
		ISLETA AMPHITHEATER:	68
		ECHO INC:	68
	Max length: 77
  2. FACILITY_KEY
	<class 'int'>
	Nulls: False
	Min: 13
	Max: 101482
	Sum: 820405773
	Mean: 46730.79135338346
	Median: 50866.0
	Standard Deviation: 32659.794578837962
	Unique values: 2835
	5 most frequent values:
		46323:	93
		65863:	85
		80949:	68
		32817:	68
		91825:	66
  3. SITE_ADDRESS
	<class 'str'>
	Nulls: True
	Unique values: 2456
	5 most frequent values:
		1601 AVENIDA CESAR CHAVEZ SE:	94
		644 OLD COORS BLVD SW:	85
		8601 CENTRAL AV NE:	77
		2200 SUNPORT BLVD SE:	75
		5210 CENTRAL AV SE:	73
	Max length: 35
  4. CITY
	<class 'str'>
	Nulls: False
	Values: ALBUQUERQUE
  5. STATE
	<class 'str'>
	Nulls: False
	Values: NM, 87
  6. ZIP
	<class 'str'>
	Nulls: False
	Unique values: 48
	5 most frequent values:
		87110

From CSVSTAT we can see the max character lengths and types of each field.

In [10]:
!csvclean -n inspection.csv

No errors.


## Problem 2 - define a database model and load the data (35 points)

Based on what you found above, create and connect to a new database, define a database table in it, and load this dataset into it.  You may use either of the methods for this step you have seen in earlier class notebooks.  You may choose to eliminate variables/columns if they are not relevant or interesting to you - explain your reasoning if you do.  Either way, you should load a majority of the columns present in the source dataset into the database and all of its rows.

Once your data has loaded successfully, run a `COUNT(*)` query to verify that all the data has loaded correctly.

Insert a combination of code and text/markdown cells here to connect to your database, define your table, load its data, and verify the loaded records.

In [11]:
%%sql
DROP TABLE IF EXISTS rest_inspec;
CREATE TABLE rest_inspec (
    facility_name CHAR(77),
    facility_key INTEGER,
    site_address CHAR(35),
    city CHAR(15),
    state CHAR(2),
    zip_address CHAR(20),
    owner_key INTEGER,
    owner_name CHAR(82),
    business_type CHAR(25),
    street_number CHAR(10),
    steet_name CHAR(24),
    street_type CHAR(4),
    direction CHAR(2),
    phone BIGINT,
    program_category CHAR(4),
    program_des CHAR(40),
    inspection_date TIMESTAMP,
    inspection_type INTEGER,
    inspection_des CHAR(32),
    serial CHAR(9),
    action_code CHAR(2),
    action_des CHAR (34),
    result_code CHAR(2),
    result_des CHAR(37),
    violation_code char(10),
    violation_des CHAR(50),
    inspection_memo INTEGER
    )

Done.
Done.


[]

I want to look at which restaurants are healthiest based on their inspection results, when their last inspection way, and if there were any violations.
I am only using the _desc fields because I do not really care about the code numbers. They seem irrelevant to the point of my table.
I also only added the zip field because it will tell me the state and city without having to have more fields.

**I at first did not include about 15 different columns, but I could not get the data to populate my table with some of the columns not there.

In [12]:
!pwd

/home/jovyan/work


In [13]:
%%sql
COPY rest_inspec FROM '/home/jovyan/work/inspection.csv'
CSV
HEADER
NULL ''
DELIMITER ',';

17556 rows affected.


[]

In [14]:
%%sql
SELECT COUNT(*) FROM rest_inspec;

1 rows affected.


count
17556


## Problem 3 - explore your data (30 points)

Now that the data is loaded, ask some questions of it!  Identify key variables of interest and note their ranges along with other useful descriptive statistics.  Choose and define a few lines of inquiry, execute queries for each, and describe what you find in your result sets.  Use any query techniques we've seen in class, including aggregate functions, transformations, subqueries, or others as appropriate. 

If you find interesting patterns, adding plots to your exploration would be useful.

Insert a combination of code and text/markdown cells here to explore your data.

In [15]:
%%sql
SELECT facility_name, COUNT(inspection_des) AS count
FROM rest_inspec
GROUP BY facility_name, violation_des
ORDER BY count DESC
LIMIT 10;

10 rows affected.


facility_name,count
SMITHS FOOD AND DRUG CENTERS INC,64
ISOTOPES PARK - OVATIONS,46
PER EHSII CECELIA GARCIA OOB LA REYNA DEL SUR,43
ISOTOPES PARK - OVATIONS,42
PER EHSII CECELIA GARCIA OOB LA REYNA DEL SUR,41
SMITHS FOOD AND DRUG CENTERS INC,36
ISLETA AMPHITHEATER,34
ISLETA AMPHITHEATER,34
TALIN MARKET WORLD FOOD FARE,27
OOB PROS RANCH MARKETS OF ALBUQUERQUE,25


The above data shows the restaurants that are inspected the most. There are some duplicates so there may be some places that have more than one location. Isleta Ampitheater looks like a duplicate, let's find out.

In [16]:
%%sql
SELECT facility_name, count(facility_name) as count
FROM rest_inspec
GROUP BY facility_name
ORDER BY count DESC
Limit 10;

10 rows affected.


facility_name,count
SMITHS FOOD AND DRUG CENTERS INC,154
ISOTOPES PARK - OVATIONS,93
PER EHSII CECELIA GARCIA OOB LA REYNA DEL SUR,85
ECHO INC,68
ISLETA AMPHITHEATER,68
DIONS PIZZA,67
CIRCLE K 8936,66
OOB CALIFORNIA DEL NORTE ICE CREAM LLC,65
TALIN MARKET WORLD FOOD FARE,64
OOB MOM THAI FOOD,61


Here we can see that Isleta was split up in the previous table from the total count. Smiths has the most inspection data on them, possibly because it is also a drug center.

In [17]:
%%sql
SELECT facility_name, COUNT(inspection_des) AS count
FROM rest_inspec
GROUP BY facility_name, violation_des
ORDER BY count ASC
LIMIT 10;

10 rows affected.


facility_name,count
IL VICINO HEIGHTS,1
OOB BLACK MESA COFFEE CO,1
MARIOS PIZZA & PASTA INC,1
OOB HAVANA RESTAURANT,1
ASIAN NOODLE BAR,1
CIRCLE K 8940,1
OOB M & M STORES,1
AMERICAN LEGION POST 72,1
OOB ISABELLAS,1
OOB 7/1/14 CARLISLE 66/SUBWAY,1


The above data shows 10 of the least inspected failities. 
There are over 50 restaurants that have only been inspected once, but to keep it clean I am only displaying 10.

In [18]:
%%sql
SELECT violation_des, COUNT(*) AS count
FROM rest_inspec
GROUP BY violation_des
ORDER BY count DESC
LIMIT 10;

10 rows affected.


violation_des,count
Additional Comments,6353
No Violations Found,3434
Delivered Informational Brochure,1657
None,1009
Plumbing: Improperly installed/maintained/supplied,492
Non-Food contact surfaces,446
Improper sanitizing,434
Food unprotected,425
"Floors, Walls, Ceilings",406
Food contact surfaces,251


The above data shows the 10 most used violation comments. The "additional comments" is used the most, but that does not give us any useful information. The next most used it "no violations found" which will give us some good information on which facilities are up to code.

In [19]:
%%sql
SELECT DISTINCT violation_code, violation_des
FROM rest_inspec
WHERE violation_des LIKE '%No%'
GROUP BY violation_code, violation_des;

10 rows affected.


violation_code,violation_des
30,Poor Hygienic Practices- No hair restraints
07 H1,No violations found at time of inspection.
04 00,No Violations Found
61,Complaint Not Substantiated
56,Notice Statement
04 30,Poor Hygienic Practices- No hair restraints
00,No Violations Found
04 61,Complaint Not Substantiated
04 35,Non-Food contact surfaces
35,Non-Food contact surfaces


We now know that No Violations Found is denoted by the code 04 00. We will use this now to find the cleanest/healthiest facilities.

In [20]:
%%sql
SELECT DISTINCT facility_name, violation_des 
FROM rest_inspec
WHERE violation_des LIKE '%No V%'
GROUP BY violation_des, facility_name
LIMIT 10;

10 rows affected.


facility_name,violation_des
OOB SALSAS GRILL,No Violations Found
OOB PROS RANCH MARKETS OF ALBUQUERQUE,No Violations Found
PIZZA HUT 012956,No Violations Found
G AND S MEAT AND FOOD COMPANY INC,No Violations Found
OOB PER WRITTEN REQUEST QUARTERS WEST RESTAURANT,No Violations Found
IL VICINO HEIGHTS,No Violations Found
OOB JUANS BROKEN TACO,No Violations Found
JAPANESE KITCHEN,No Violations Found
SHARK REEF RESTAURANT,No Violations Found
GIANT 6005,No Violations Found


There are over 1705 rows that had no violations. I have limited it to 10. I could not get using the codes, so I did it by violation_des.

## Bonus (10 points)

We have seen a few simple techniques for normalizing data in class.  Choose a column or set of columns that might be redundant and normalize them out to another table:

* Identify the columns to normalize in the source data table
* Create one or more new tables to hold the normalized records
* Extract data into the new tables
* Re-insert the new foreign key references from the new table into the source table

In [21]:
%%sql
DROP TABLE IF EXISTS facility_norm;
CREATE TABLE facility_norm AS
    SELECT facility_name, facility_key, site_address, city, state, zip_address 
    FROM rest_inspec;

Done.
17556 rows affected.


[]

In [22]:
%%sql
DROP TABLE IF EXISTS owner_norm;
CREATE TABLE owner_norm AS
    SELECT owner_key, owner_name, facility_key, business_type, street_number, steet_name, street_type, direction, phone, program_category, program_des
    FROM rest_inspec;

Done.
17556 rows affected.


[]

In [23]:
%%sql
DROP TABLE IF EXISTS inspection_norm;
CREATE TABLE inspection_norm AS
    SELECT inspection_date, inspection_type, facility_key, inspection_des, serial, action_code, action_des, result_code, result_des, violation_code, violation_des, inspection_memo
    FROM rest_inspec;

Done.
17556 rows affected.


[]

I have created 3 normalized tables: facility table, owner table, and inspection table. All the tables are connected by the facility key attribute. Each inspection is done on one facility, and each facility has an owner. The facility_key attribute is the primary key for the facility table and the foreign key for the other two tables.

I am not sure what is meant by reinserting the foreign keys into the source data. I now have my foreign keys in my 3 normalized tables.

In [ ]:
%%sql
SELECT facility_name, owner_name, inspection_memo
FROM facility_norm, owner_norm, inspection_norm
WHERE facility_norm.facility_key = owner_norm.facility_key
AND facility_norm.facility_key = inspection_norm.facility_key;

Each time I run the above code, the kernel dies. I am not sure why, the code is correct and it should run properly.